In [45]:
import pandas as pd
import numpy as np
import re

# 데이터 불러오기
df = pd.read_csv("data/merge_data.csv", low_memory=False)

# 중복제거
df.drop_duplicates(inplace=True)

In [47]:
# 텍스트 컬럼 특수문자 정리
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r"[\\n\\r\\t]", " ", text)
        text = re.sub(r"[^가-힣a-zA-Z0-9.,:()\\s]", "", text)
        return text.strip()
    return text

text_cols = ["Primary Type", "Description", "Location Description"]
for col in text_cols:
    df[col] = df[col].apply(clean_text)

In [49]:
# 날짜 형식 정리 + 연/월 컬럼 추가
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month

/var/folders/1k/kfyrfwj51r7_zts1bndgk56c0000gn/T/ipykernel_96490/171308769.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")


In [51]:
# Community Area, District 결측치 처리
df["Community Area"] = df["Community Area"].fillna("Unknown")
df["District"] = df["District"].fillna("Unknown")

In [53]:
# 위도/경도 숫자형으로 변환
df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

In [55]:
# 숫자인 District만 평균으로 보정
numeric_districts = df["District"].apply(lambda x: str(x).isdigit())

df.loc[numeric_districts, "Latitude"] = (
    df[numeric_districts]
    .groupby("District")["Latitude"]
    .transform(lambda x: x.fillna(x.mean()))
)

df.loc[numeric_districts, "Longitude"] = (
    df[numeric_districts]
    .groupby("District")["Longitude"]
    .transform(lambda x: x.fillna(x.mean()))
)

In [57]:
# 남은 결측치는 0으로 보정
df["Latitude"] = df["Latitude"].fillna(0)
df["Longitude"] = df["Longitude"].fillna(0)

In [59]:
# 체포 여부 문자열 처리
df["Arrest"] = df["Arrest"].map({True: "체포됨", False: "체포되지 않음"}).fillna("정보없음")

In [61]:
# 필요한 컬럼만 선택
columns_to_keep = [
    "Date", "Block", "Primary Type", "Description", "Location Description", 
    "Arrest", "Beat", "District", "Community Area", "Latitude", "Longitude"
]
df = df[columns_to_keep]

In [63]:
# text 생성
df["text"] = (
    df["Date"].dt.strftime('%Y-%m-%d') + " / " +
    df["Block"].astype(str) + "에서 발생한 범죄: " +
    df["Primary Type"].astype(str) + " (" +
    df["Description"].astype(str) + "). 장소는 " +
    df["Location Description"].astype(str) + "이며, " +
    df["Arrest"] + ". 관할: Beat " + df["Beat"].astype(str) +
    ", District " + df["District"].astype(str) +
    ". 지역코드: " + df["Community Area"].astype(str)
)

In [ ]:
# 저장
final_columns = ["text"] + columns_to_keep
df[final_columns].to_csv("rag_ready_data.csv", index=False, encoding="utf-8-sig")

print("전처리 완료! 저장된 파일: rag_ready_data.csv")

In [69]:
df.isnull().sum()

Date                       0
Block                      0
Primary Type               0
Description                0
Location Description    1968
Arrest                     0
Beat                       0
District                   0
Community Area             0
Latitude                   0
Longitude                  0
text                       0
dtype: int64